In [1]:
%load_ext autoreload
%autoreload 2

import yeahml as yml

## Set up

First, we need to make sure a configuration file has been created and follows the reccomended format

In [2]:
!cat ./examples/cats_v_dogs/main_config.yaml

# all paths are relative to yaml/execution?
# TODO: naming/structure will need to be improved
meta:
  name: 'cats_v_dogs'
  experiment_dir: 'trial_01'
  saver:
    save_params_name: "best_params_saver"

logging:
  console:
    level: 'info'
    format_str: null
  file:
    level: 'debug'
    format_str: null
  graph_spec: True

performance:
  loss_fn: 
    type: 'binary_crossentropy'
  type: ["BinaryAccuracy"]

data:
  in: 
    dim: [150, 150, 3]
    dtype: 'float32'
  label:
    dim: [1]
    dtype: 'int64'
  TFR_parse:
    feature:
      name: "/image"
      dtype: "float32"
      tftype: "fixedlenfeature"
      in_type: "string"
    label:
      name: "/label"
      dtype: "int64" # what to decode it to
      tftype: "fixedlenfeature"
      in_type: "int64" # what it is encoded as
  TFR:
    dir: './examples/cats_v_dogs/data/record_holder/150'
    train: 'train.tfrecords'
    validation: 'val.tfrecords'
    test: 'test.tfrecords'
  # image:
  #   standardize: True
  #   augmentation:

## Parse configuration files

Next, we will parse out the relevant bits of information into modular components.

In [3]:
## Core Logic
example = "./examples/cats_v_dogs/main_config.yaml"
config_dict = yml.create_configs(example)

./examples/cats_v_dogs/trial_01 created
./examples/cats_v_dogs/trial_01/best_params created
./examples/cats_v_dogs/trial_01/tf_logs created
./examples/cats_v_dogs/trial_01/yf_logs created


In [4]:
meta_cdict = config_dict["meta"]
log_cdict = config_dict["logging"]
perf_cdict = config_dict["performance"]
data_cdict = config_dict["data"]
hp_cdict = config_dict["hyper_parameters"]
model_cdict = config_dict["model"]

## Examine configs

Each of these configs is a simple python dictionary with the expected format for yeahml.

They can be extracted/created seperately if desired (an example of modifying the `model_cdict` is shown next)

In [5]:
config_dict

{'meta': {'name': 'cats_v_dogs',
  'experiment_dir': 'trial_01',
  'save_params': 'best_params_saver',
  'log_dir': './examples/cats_v_dogs/trial_01',
  'save_weights_path': './examples/cats_v_dogs/trial_01/best_params/best_params_saver.h5',
  'best_params': './examples/cats_v_dogs/trial_01/best_params',
  'tf_logs': './examples/cats_v_dogs/trial_01/tf_logs',
  'yf_logs': './examples/cats_v_dogs/trial_01/yf_logs'},
 'logging': {'log_c_lvl': 'INFO',
  'log_c_str': '%(name)-12s: %(levelname)-8s %(message)s',
  'log_f_lvl': 'DEBUG',
  'log_f_str': '[%(asctime)s - %(filename)s:%(lineno)s - %(funcName)20s()][%(levelname)-8s]: %(message)s',
  'log_g_lvl': 'DEBUG',
  'log_g_str': '%(name)-12s: %(levelname)-8s %(message)s',
  'log_p_lvl': 'DEBUG',
  'log_p_str': '[%(levelname)-8s] %(message)s'},
 'performance': {'loss_fn': {'type': 'binary_crossentropy'},
  'met_list': ['binaryaccuracy'],
  'met_opts_list': None},
 'data': {'input_layer_dim': [150, 150, 3],
  'in_dim': [None, 150, 150, 3],
  '

## Build the model
(according to our specifications)

In [6]:
model = yml.build_model(meta_cdict, model_cdict, log_cdict, data_cdict)

build_logger: INFO     -> START building graph
build_logger: INFO     -> START building hidden block
graph_logger: INFO     | batch_norm_0    | no_shape
graph_logger: INFO     | conv_1          | no_shape
graph_logger: INFO     | batch_norm_1    | no_shape
graph_logger: INFO     | pool_1          | no_shape
graph_logger: INFO     | conv_2          | no_shape
graph_logger: INFO     | batch_norm_2    | no_shape
graph_logger: INFO     | pool_2          | no_shape
graph_logger: INFO     | conv_3          | no_shape
graph_logger: INFO     | batch_norm_3    | no_shape
graph_logger: INFO     | pool_3          | no_shape
graph_logger: INFO     | dropout_1       | no_shape
graph_logger: INFO     | flatten_1       | no_shape
graph_logger: INFO     | dense_1         | no_shape
graph_logger: INFO     | dropout_2       | no_shape
graph_logger: INFO     | batch_norm_4    | no_shape
graph_logger: INFO     | dense_2         | no_shape
graph_logger: INFO     | dense_out       | no_shape
build_logger: I

In [7]:
# this is a standard tf2 model
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
batch_norm_0 (BatchNormaliza (None, 150, 150, 3)       12        
_________________________________________________________________
conv_1 (Conv2D)              (None, 74, 74, 16)        448       
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 74, 74, 16)        64        
_________________________________________________________________
pool_1 (AveragePooling2D)    (None, 37, 37, 16)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 35, 35, 32)        4640      
_________________________________________________________________
batch_norm_2 (BatchNormaliza (None, 35, 35, 32)        128   

## Train the model

In [8]:
train_dict = yml.train_model(
    model, meta_cdict, log_cdict, data_cdict, hp_cdict, perf_cdict
)

train_logger: INFO     -> START training graph
train_logger: INFO     start creating train_dict
train_logger: INFO     [END] creating train_dict


In [9]:
# the returned train_dict contains relevant information from training
print(train_dict)

{'train_losses': [0.68455386, 0.63612455, 0.6074224], 'val_losses': [0.6097455, 0.5796257, 0.6080216], 'epochs': [0, 1, 2], 'binaryaccuracy': 0.6684375, 'val_binaryaccuracy': 0.6618125}


## Evaluate Model

In [10]:
eval_dict = yml.eval_model(
    model,
    meta_cdict,
    log_cdict,
    data_cdict,
    hp_cdict,
    perf_cdict,
)
print(eval_dict)

eval_logger : INFO     params loaded from ./examples/cats_v_dogs/trial_01/best_params/best_params_saver.h5
eval_logger : INFO     -> START evaluating model
eval_logger : INFO     [END] evaluating model
eval_logger : INFO     -> START creating eval_dict
eval_logger : INFO     [END] creating eval_dict


{'binaryaccuracy': 0.7010625}
